In [2]:
class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)

    def _traverse_dict(self, instance_dict:dict):
        response = {}
        for key, value in instance_dict.items():
            response[key] = self._traverse(key, value)
        return response

    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        else:
            return value

from dataclasses import dataclass

@dataclass
class BinaryTree(ToDictMixin):
    value: object
    left: 'BinaryTree' = None
    right: 'BinaryTree' = None

class IndexableNode(BinaryTree):
    def _traverse(self):
        if self.left:
            yield from self.left._traverse()
        yield self
        if self.right:
            yield from self.right._traverse()

    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if index == i:
                return item.value
        raise Exception("Index Error Tree has less elements")

class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass 
        return count

 
tree = IndexableNode(10, IndexableNode(7, right=IndexableNode(9)), IndexableNode(13, left=IndexableNode(11)))
print(tree[3])
print(11 in tree)

11
True


In [2]:
from pprint import pprint


class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left, right)
        self.parent = parent

    def _traverse(self, key, value):
        if isinstance(value, BinaryTreeWithParent) and key == "parent":
            return value.value
        else:
            return super()._traverse(key, value)


root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
root.right = BinaryTreeWithParent(13, parent=root)
root.right.left = BinaryTreeWithParent(11, parent=root.right)
pprint(root.to_dict(), indent=2, width=20)


{ 'left': { 'left': None,
            'parent': 10,
            'right': { 'left': None,
                       'parent': 7,
                       'right': None,
                       'value': 9},
            'value': 7},
  'parent': None,
  'right': { 'left': { 'left': None,
                       'parent': 13,
                       'right': None,
                       'value': 11},
             'parent': 10,
             'right': None,
             'value': 13},
  'value': 10}


In [3]:
class NamedSubTree(ToDictMixin):
    def __init__(self, name, tree_with_parent):
        self.name = name
        self.tree_with_parent = tree_with_parent


import inspect

my_tree = NamedSubTree("foobar", root.left.right)
my_tree.to_dict()
print(inspect.getsource(my_tree._traverse))


    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        else:
            return value



In [4]:
import json


class JSONMixin:
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)

    def to_json(self):
        return json.dumps(self.to_dict())


class Switch(ToDictMixin, JSONMixin):
    def __init__(self, name, state):
        self.name = name
        self.state = state


class Machine(ToDictMixin, JSONMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk or []


class DCRack(ToDictMixin, JSONMixin):
    def __init__(self, switches=None, machines=None):
        self.switches = switches or []
        self.machines = [Machine(**kwargs) for kwargs in machines]


serialized = """{
"switches": {"ports": 5, "speed": 1e9},
"machines": [
{"cores": 8, "ram": 32e9, "disk": 5e12},
{"cores": 4, "ram": 16e9, "disk": 1e12},
{"cores": 2, "ram": 4e9, "disk": 500e9}
]
}"""

deserialized = DCRack.from_json(serialized)
print(deserialized)

In [5]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10

    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field
    
    
    def get_private_field(self):
        return self.__private_field

In [6]:
c = MyObject()
MyObject.get_private_field_of_instance(c)
# c.__private_field

10

In [7]:
class ChildPrivateClass(MyObject):
    def get_private_field(self):
        return self._MyObject__private_field

c = ChildPrivateClass()
c.get_private_field()

10

In [8]:
def coroutine():
    for i in range(1, 10):
        print("From generator {}".format((yield i)))
c = coroutine()
c.send(None)
# c.send(None)

1

In [9]:
# c.send(None)
for i in range(3):
    print(c.send(i))

From generator 0
2
From generator 1
3
From generator 2
4


In [10]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

class BetterNode(SequenceNode, Sequence):
    pass

In [11]:
from collections import defaultdict

class StudentRecord:
    __percentage_dict = defaultdict(int)

    def __init__(self, name, number):
        self.name = name
        self.number = number
    
    def get_percentage(self):
        return self.__percentage_dict[self.number]

    def set_percentage(self, percentage):
        if not isinstance(percentage, (int, float)):
            raise ValueError("Percentage should be an real postive number")
        if percentage < 0 or percentage > 100:
            raise ValueError("Percentage should be between 0 and 100")
        self.__percentage_dict[self.number] = percentage


s1 = StudentRecord("Student1", 1)
s1.set_percentage(90)
print(s1.get_percentage())

s1.set_percentage(s1.get_percentage() - 1)
print(s1.get_percentage())
print(s1._StudentRecord__percentage_dict)

90
89
defaultdict(<class 'int'>, {1: 89})


In [12]:
class A:
    @property
    def a(self):
        return 1
    
    @a.setter
    def a(self, value):
        print("Setting value to {}".format(value))


class B(A):
    pass

In [13]:
c = B()
c.a = 10
c.a

Setting value to 10


1

In [14]:
from weakref import WeakKeyDictionary

class PercentageValidator:

    def __init__(self):
        self.__value = 0 

    def __get__(self, instance, instance_klass):
        return self.__value
    
    def __set__(self, instance, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Percentage should be an real postive number")
        if value < 0 or value > 100:
            raise ValueError("Percentage should be between 0 and 100")
        self.__value = value


class Constant:
    def __init__(self):
        self.__value = None

    
    def __get__(self, instance, instance_klass):
        return self.__value

    def __set__(self, instance, value):
        if self.__value is None:
            self.__value = value
            return 
        raise AttributeError("Can't set attribute")


class StudentRecord:
    english = PercentageValidator()
    math = PercentageValidator()
    physics = PercentageValidator()
    number = Constant()
    

    def __init__(self, name, number):
        self.name = name

s1 = StudentRecord("Student1", 10)
s2 = StudentRecord("Student2", 20)
s1.physics = 100
s2.physics = 99
s1.math = 100
s2.math = 98
print(s1.math, s2.math)

98 98


In [15]:
# creating a constant validator using Descriptor
class Constant:
    def __init__(self):
        self.__value = None
    
    def __get__(self, instance, instance_klass):
        if instance is None:
            return self
        return self.value

    def __set__(self, instance, value):
        if self.__value is None:
            self.__value = value

        raise AttributeError("Can't set attribute")

In [16]:
from collections import defaultdict
from weakref import WeakKeyDictionary

class PercentageValidator:

    def __init__(self):
        self.__value = WeakKeyDictionary()

    def __get__(self, instance, instance_klass):
        return self.__value.get(instance, 0)
    
    def __set__(self, instance, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Percentage should be an real postive number")
        if value < 0 or value > 100:
            raise ValueError("Percentage should be between 0 and 100")
        self.__value[instance] = value

    def __getattr__(self, name):
        print("Get attribute", name)
        return self.__value.get(name, None)

    def __getattribute__(self, __name: str):
        print("Get attribute", __name)
        return super().__getattribute__(__name)

class Constant:
    def __init__(self):
        self.__value = {}
    
    def __get__(self, instance, instance_klass):
        print("Get attribute get ", instance)
        return self.__value.get(instance, None)

    def __set__(self, instance, value):
        current_value = self.__value.get(instance, None)
        if current_value is None:
            self.__value[instance] = value
            return 
        raise AttributeError("Can't set attribute")

    def __getattr__(self, name):
        print("Get attribute", name)
        return self.__value.get(name, None)

    def __getattribute__(self, __name: str):
        print("Get attribute", __name)
        return super().__getattribute__(__name)

class StudentRecord:
    english = PercentageValidator()
    math = PercentageValidator()
    physics = PercentageValidator()
    number = Constant()

    def __init__(self, name, number):
        self.name = name
    
    def __getattr__(self, name):
        print("Get attribute", name)

        return self.__value.get(name, None)

    def __getattribute__(self, __name: str):
        print("Get attribute", __name)
        return super().__getattribute__(__name)

s1 = StudentRecord("Student1", 10)
s2 = StudentRecord("Student2", 20)
s1.physics = 100
s2.physics = 99
s1.math = 100
s2.math = 98
s1.math = s1.math - 10
print(s1.math, s2.math)

Get attribute _PercentageValidator__value
Get attribute _PercentageValidator__value
Get attribute _PercentageValidator__value
Get attribute _PercentageValidator__value
Get attribute math
Get attribute _PercentageValidator__value
Get attribute _PercentageValidator__value
Get attribute math
Get attribute _PercentageValidator__value
Get attribute math
Get attribute _PercentageValidator__value
90 98


In [17]:
StudentRecord.__dict__['english'].__set__(s1, 80)

Get attribute __set__
Get attribute _PercentageValidator__value


In [18]:
s1.name

Get attribute name


'Student1'

In [19]:
class LazyRecord:
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = "Value for {}".format(name)
        setattr(self, name, value)
        print("Will not called")
        return value 
    
    def __getattribute__(self, __name: str):
        print("Called __getattribute__")
        try:
            value = super().__getattribute__(__name)
            print("Value found", value)
            return value
        except AttributeError:
            response = self.__getattr__(__name)
            print("Called once")
            return response

class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print("Called __getattr__({})".format(name))
        return super().__getattr__(name)

In [20]:
c = LoggingLazyRecord()
c.name
print("Before: ", c.__dict__)

Called __getattribute__
Called __getattribute__
Value found <bound method LoggingLazyRecord.__getattr__ of <__main__.LoggingLazyRecord object at 0x7ff26e72acd0>>
Called __getattr__(name)
Will not called
Called once
Called __getattribute__
Value found {'exists': 5, 'name': 'Value for name'}
Before:  {'exists': 5, 'name': 'Value for name'}


Called __getattribute__
Value found Value for name


'Value for name'

In [1]:
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None

    def __get__(self, instance, instance_type):
        print("Inside get", instance, instance_type)

        if instance is None:
            return None
        return getattr(instance, self.internal_name, "")

    def __set_name__(self, owner, name):
        print(owner, name, self)
        self.name = name
        self.internal_name = "_" + name

    def __set__(self, instance, value):
        print("INSTANCE", instance)
        setattr(instance, self.internal_name, value)


class Meta(type):
    def __new__(meta, name, bases, klass_dict: dict):
        # print(meta, bases, klass_dict)
        for key, value in klass_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = "_" + key
        cls = type.__new__(meta, name, bases, klass_dict)
        return cls


class DatabaseRow(metaclass=Meta):
    def __init_subclass__(cls) -> None:
        # print("super cls", cls)
        return super().__init_subclass__()


class Customer:
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()


cust = Customer()
print(type(cust.first_name))
cust.first_name = "Eduardo"
cust.last_name = "Perez"
print(cust.__dict__)

from functools import wraps


def trace_func(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if hasattr(func, "tracing"):
            return func

        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(f"{func.__name__}({args}, {kwargs}) -> {result}")

    wrapper.tracing = True
    return wrapper


class TraceDict(dict):
    @trace_func
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


    @trace_func
    def __setitem__(self, *args, **kwargs):
        super().__setitem__(*args, **kwargs)

    @trace_func
    def __getitem__(self, *args, **kwargs):
        return super().__getitem__(*args, **kwargs)


import types

trace_types = (
    types.MethodType,
    types.FunctionType,
    types.BuiltinMethodType,
    types.BuiltinFunctionType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType,
)
dir


class TraceMeta(type):
    def __new__(meta, name, bases, klass_dict: dict):
        klass = type.__new__(meta, name, bases, klass_dict)

        for key in dir(klass):
            value = getattr(klass, key)
            if isinstance(value, trace_types):
                setattr(klass, key, trace_func(value))
        return klass


class TraceDict(dict, metaclass=TraceMeta):
    pass


def trace(klass):
    for key in dir(klass):
        value = getattr(klass, key)
        if isinstance(value, trace_types):
            setattr(klass, key, trace_func(value))
    return klass


@trace
class TraceDict(dict):
    pass


class OtherMeta(type):
    pass


@trace
class Trace(dict, metaclass=OtherMeta):
    pass


trace_dict = TraceDict([("a", 1), ("b", 2)])
trace_dict["c"] = 3
trace_dict["a"]


<class '__main__.Customer'> first_name <__main__.Field object at 0x7f9a1e771dd0>
<class '__main__.Customer'> last_name <__main__.Field object at 0x7f9a1e7715d0>
<class '__main__.Customer'> prefix <__main__.Field object at 0x7f9a1e772090>
<class '__main__.Customer'> suffix <__main__.Field object at 0x7f9a1e772190>
Inside get <__main__.Customer object at 0x7f9a1e772390> <class '__main__.Customer'>
<class 'str'>
INSTANCE <__main__.Customer object at 0x7f9a1e772390>
INSTANCE <__main__.Customer object at 0x7f9a1e772390>
{'_first_name': 'Eduardo', '_last_name': 'Perez'}
__format__(({}, ''), {}) -> {}
__new__((<class '__main__.TraceDict'>, [('a', 1), ('b', 2)]), {}) -> {}
__getitem__(({'a': 1, 'b': 2, 'c': 3}, 'a'), {}) -> 1


1

In [38]:
from dataclasses import dataclass


class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        response = {}
        for key, value in instance_dict.items():
            response[key] = self._traverse(key, value)
        return response
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, "__dict__"):
            return self._traverse_dict(value.__dict__)
        else:
            return value


@dataclass
class BinaryTree(ToDictMixin):
    value: object
    left: 'BinaryTree' = None
    right: 'BinaryTree' = None


class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left, right)
        self.parent = parent
    
    def _traverse(self, key, value):
        if isinstance(value, self.__class__) and key == "parent":
            return value.value
        return super()._traverse(key, value)

# class IndexableTree(BinaryTree):
#     def _intraverse(self):
#         if self.left:
#             yield from self.left._traverse()
#         yield self
#         if self.right:
#             yield from self.right._traverse()

#     def __getitem__(self, index):
#         for i, item in enumerate(self._traverse(self.value)):
#             if i == index:
#                 return item
#         raise IndexError("Index out of range")

# # tree = BinaryTree(10, left=BinaryTree(7, right=BinaryTree(9)), right=BinaryTree(13, left=BinaryTree(11)))
# from pprint import pprint

# tree = IndexableTree(10, IndexableTree(7, right=IndexableTree(9)), IndexableTree(13, left=IndexableTree(11)))
# # pprint(tree.to_dict())
# # print(list(tree.)
# # how to access private variable of a class
# print(tree._IndexableTree__dict__)
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
root.right = BinaryTreeWithParent(13, parent=root)
root.right.left = BinaryTreeWithParent(11, parent=root.right)
pprint(tree.to_dict(), width=20)


<class 'int'> False
<class '__main__.BinaryTreeWithParent'> False
<class 'int'> False
<class 'NoneType'> False
<class '__main__.BinaryTreeWithParent'> False
<class 'int'> False
<class 'NoneType'> False
<class 'NoneType'> False
<class 'NoneType'> True
<class 'NoneType'> True
<class '__main__.BinaryTreeWithParent'> False
<class 'int'> False
<class '__main__.BinaryTreeWithParent'> False
<class 'int'> False
<class 'NoneType'> False
<class 'NoneType'> False
<class 'NoneType'> True
<class 'NoneType'> False
<class 'NoneType'> True
<class 'NoneType'> True
{'left': {'left': None,
          'parent': None,
          'right': {'left': None,
                    'parent': None,
                    'right': None,
                    'value': 9},
          'value': 7},
 'parent': None,
 'right': {'left': {'left': None,
                    'parent': None,
                    'right': None,
                    'value': 11},
           'parent': None,
           'right': None,
           'value': 13},
 

In [40]:
from dataclasses import dataclass


class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        response = {}
        for key, value in instance_dict.items():
            response[key] = self._traverse(key, value)
        return response
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, "__dict__"):
            return self._traverse_dict(value.__dict__)
        else:
            return value


@dataclass
class BinaryTree(ToDictMixin):
    value: object
    left: 'BinaryTree' = None
    right: 'BinaryTree' = None


class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left, right)
        self.parent = parent
    
    def _traverse(self, key, value):
        if isinstance(value, self.__class__) and key == "parent":
            return value.value
        return super()._traverse(key, value)

root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
root.right = BinaryTreeWithParent(13, parent=root)
root.right.left = BinaryTreeWithParent(11, parent=root.right)
pprint(root.to_dict(), width=20)


{'left': {'left': None,
          'parent': 10,
          'right': {'left': None,
                    'parent': 7,
                    'right': None,
                    'value': 9},
          'value': 7},
 'parent': None,
 'right': {'left': {'left': None,
                    'parent': 13,
                    'right': None,
                    'value': 11},
           'parent': 10,
           'right': None,
           'value': 13},
 'value': 10}
